In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import time
import numpy as np
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import src
from src.reload import deep_reload
from src.utilities import load_itineraries

In [2]:
#Importing itineraries
itineraries = load_itineraries(weekday=True)  # Change to False for weekend

Loaded itineraries from weekday file.


In [ ]:
deep_reload(src)
n=1
print(n)

solver_kwargs = {'_name': 'cbc', 'executable': '/opt/homebrew/bin/cbc'}

itinerary_kwargs={
    'initial_soc':1,
    'final_soc':1,
    'initial_sof':1,
    'final_sof':1,
    'battery_capacity':82*3.6e6,
    'home_charger_likelihood':1,
    'work_charger_likelihood':1,
    'destination_charger_likelihood':.1,
}

# itinerary_kwargs={}
problem=src.optimization.EVCSP(itineraries[n],itinerary_kwargs=itinerary_kwargs)
problem.Solve(solver_kwargs)
print(f'solver status: {problem.solver_status}, '+
      f'solver termination condition: {problem.solver_termination_condition}')
print(f'sic: {problem.sic}')

In [ ]:
deep_reload(src)

line_kwargs={
    'linewidth':3,
}

axes_kwargs={
    'facecolor':'whitesmoke',
    'ylim':[0,1],
    'xlabel':'Trip/Park Event',
    'ylabel':'SOC [-]',
}

fig,ax=plt.subplots(1,1,figsize=(8,4))
if not hasattr(ax,'__iter__'):
    ax=[ax]
    
ax[0].plot(problem.solution.soc)
# ax[0].plot(problem.solution.sof)
# ax[0].plot(problem.solution.u_cd_cs[0])

_=[ax.grid(ls='--') for ax in ax]

In [3]:
deep_reload(src)

itinerary_kwargs={
	'tiles':5,
	'initial_sof':1,
	'final_sof':1,
    'battery_capacity':82*3.6e6,
	'home_charger_likelihood':1,
	'work_charger_likelihood':0.75,
	'destination_charger_likelihood':.25
}

experiment_keys=[
    'battery_capacity'
	'home_charger_likelihood',
	'work_charger_likelihood',
	'destination_charger_likelihood',
]

experiment_levels=[
    [75*3.6e6, 66*3.6e6, 60*3.6e6],
	[0.5,.75, 1],
	[0.25,0.5,0.75],
	[0,.25,.5],
]

experiment_inputs=src.experiments.GenerateExperiment(
	itinerary_kwargs,experiment_keys,experiment_levels)

exp_itineraries=src.experiments.SelectItineraries(itineraries,1,seed=1000)

solver_kwargs = {'_name': 'cbc', 'executable': '/opt/homebrew/bin/cbc','threads':10}

vehicle_classes=[
    src.optimization.EVCSP,
]

experiment_results=src.experiments.RunExperiment(
    vehicle_classes,
	exp_itineraries,
    experiment_inputs,
    iter_time=1,
    solver_kwargs=solver_kwargs,
    disp=False,
    repititions=3,
)

In [ ]:
experiment_results=pkl.load(open('Data/Generated_Data/output_12072023_174102.pkl','rb'))

In [ ]:
deep_reload(src)

out=src.experiments.ProcessResults(experiment_results)
out

In [ ]:
out['sic_bev']=out['sic_mean_0']
out['bc']=out['battery_capacity']
out['hcl']=out['home_charger_likelihood']
out['wcl']=out['work_charger_likelihood']
out['dcl']=out['destination_charger_likelihood']

formula_bev=(
    'sic_bev~'+
    'bc*'+
    'hcl*'+
    'wcl*'+
    'dcl'
)
model_bev=smf.ols(formula=formula_bev,data=out).fit()


In [ ]:
deep_reload(src)

cmap=src.figures.ReturnColorMap('viridis')
cs=src.figures.colors['forest']

bar_kwargs={
    'fc':cs[3],
    'ec':cs[2],
    'lw':5,
    'error_kw':{
        'ecolor':cs[1],
        'capsize':10,
        'capthick':5,
        'lw':5,
    }}

axes_kwargs={
    'facecolor':'whitesmoke',
    # 'xlim':[-.8,.8],
}

alpha=.05

fig,ax=plt.subplots(1,3,figsize=(15,5))

src.figures.FactorsPlot(model_bev,ax=ax[0],alpha=alpha,
                        bar_kwargs=bar_kwargs,axes_kwargs=axes_kwargs)

_=[ax.grid(ls='--') for ax in ax]